In [1]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/COVID-CXR-Image-Classification-Project/research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/COVID-CXR-Image-Classification-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    model_path: Path
    epochs: int
    imgsz: int
    batch_size: int
    lr: float
    weights: str
    num_class: int
    include_top: bool

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])

    def get__training_config(self) -> TrainingConfig:
        config = self.config.training 
        params = self.params
        create_directories([config.root_dir])
        training_config = TrainingConfig(
            root_dir = config.root_dir,
            epochs = params.EPOCHS,
            imgsz=params.IMG_SZ,
            include_top=params.INCLUDE_TOP,
            batch_size=params.BATCH_SIZE,
            weights=params.WEIGHTS,
            lr=params.LEARNING_RATE,
            model_path=Path(config.trained_model_path),
            num_class=params.CLASSES
        )
        return training_config

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-01-24 12:55:57.369944: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 12:56:01.860495: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-24 12:56:10.244914: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config 

    def train_val_gen(self):
        # Set your dataset directory
        train_dir = os.path.join("artifacts", "data_preprocessing", "split_dataset", "train")
        val_dir = os.path.join("artifacts", "data_preprocessing", "split_dataset", "val")
        

        # Use ImageDataGenerator for data augmentation and normalization
        train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                           zoom_range=0.2, horizontal_flip=True)
        val_datagen = ImageDataGenerator(rescale=1./255)

        # Create data generators
        train_generator = train_datagen.flow_from_directory(train_dir, target_size=(self.config.imgsz, self.config.imgsz),
                                                            batch_size=self.config.batch_size, class_mode='categorical')

        val_generator = val_datagen.flow_from_directory(val_dir, target_size=(self.config.imgsz, self.config.imgsz),
                                                        batch_size=self.config.batch_size, class_mode='categorical')

        # Set the number of training steps
        return train_generator, val_generator
    
    def train(self):
        pretrained_model = tf.keras.applications.VGG16(weights=self.config.weights, 
                                                       include_top=self.config.include_top, 
                                                       input_shape=[self.config.imgsz, self.config.imgsz, 3])
        pretrained_model.trainable=False
        vgg16_model = tf.keras.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        train_generator, val_generator = self.train_val_gen()
    
        steps_per_epoch = train_generator.samples // self.config.batch_size
        validation_steps = val_generator.samples // self.config.batch_size

        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(self.config.model_path, save_best_only=True)

        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",patience=5, restore_best_weights=True
        )

        vgg16_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
        history = vgg16_model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.config.epochs,
                            validation_data=val_generator, validation_steps=validation_steps,callbacks=[checkpoint_callback,early_stopping_callback])

In [18]:
try:
    config = ConfigurationManager()
    train_config = config.get__training_config()
    training = Training(config=train_config)
    training.train()
except Exception as e:
    raise e

[2024-01-24 13:15:03,408: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-24 13:15:03,414: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-24 13:15:03,416: INFO: common: created directory at: artifacts]
[2024-01-24 13:15:03,419: INFO: common: created directory at: artifacts/training]


Found 1188 images belonging to 3 classes.
Found 342 images belonging to 3 classes.


2024-01-24 13:15:08.034978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-01-24 13:15:11.216860: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-24 13:15:12.015495: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-01-24 13:15:13.903287: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-01-24 13:15:13.963301: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-01-24 13:15:14.864643: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10

74/74 [==============================] - ETA: 0s - loss: 0.8957 - accuracy: 0.6195 

2024-01-24 13:38:36.744999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


74/74 [==============================] - 1772s 24s/step - loss: 0.8957 - accuracy: 0.6195 - val_loss: 0.7164 - val_accuracy: 0.7649
